In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
bitstamp_file = "resources/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv"
bitstamp_df = pd.read_csv(bitstamp_file)
bitstamp_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
coinbase_file = "resources/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv"
coinbase_df = pd.read_csv(coinbase_file)
coinbase_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1417411980,300.0,300.0,300.0,300.0,0.01,3.0,300.0
1,1417412040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1417412100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1417412160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1417412220,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Transform bitstamp DataFrame

In [5]:
# Create a copy of the dataframe
bitstamp_transform = bitstamp_df.copy()

# Drop NaN rows
bitstamp_transform = bitstamp_transform.dropna(thresh=2)

# Rename columns to acceptable values within postgres
bitstamp_transform.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume_btc', 'volume_currency', 'weighted_price']

# Create a new index to as the id
# bitstamp_transform = bitstamp_transform.reset_index()
# bitstamp_transform.columns['index'] = 'id'
bitstamp_transform['id'] = bitstamp_transform.index

# Display database
bitstamp_transform

,timestamp,open,high,low,close,volume_btc,volume_currency,weighted_price,id
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000,0
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000,478
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411,547
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000,548
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000,1224
...,...,...,...,...,...,...,...,...,...
3997692,1565567760,11555.57,11555.57,11540.37,11540.58,0.036868,425.909106,11552.336234,3997692
3997693,1565567820,11553.49,11556.22,11553.49,11556.22,0.623462,7204.428272,11555.520505,3997693
3997694,1565567880,11559.73,11561.22,11546.77,11561.22,0.159070,1838.731403,11559.252199,3997694
3997695,1565567940,11559.73,11589.73,11528.73,11528.73,16.198210,187504.635170,11575.638889,3997695


### Transform coinbase DataFrame

In [ ]:
# Create a copy of the dataframe
coinbase_transform = coinbase_df.copy()

# Drop NaN rows
coinbase_transform = coinbase_transform.dropna(thresh=2)

# Rename columns to acceptable values within postgres
coinbase_transform.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume_btc', 'volume_currency', 'weighted_price']
coinbase_transform

### Create database connection

In [ ]:
connection_string = "postgres:password@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

### Load Dataframes into database

In [ ]:
bitstamp_transform.to_sql(name='bitstamp', con=engine, if_exists='append')

In [ ]:
coinbase_transform.to_sql(name='coinbase', con=engine, if_exists='append', index=True)